# Pattern Matching 

Pattern matching is a way of capturing value from inside a pattern.
Consider for exemple:

In [7]:
using Pkg
Pkg.activate(".")
using MLStyle

  Activating project at `~/Main/EMAp/PhDThesis/notes/FunctionalProgrammingCategoryTheory/julia`


In [8]:
x, y, z = 1,2,3

(1, 2, 3)

This can be thought as a very simple pattern matching. We are assigning the values of our variables
based on the order of the inputs. Here is another simple example.

In [42]:
@show x,y,z = [0,2,1]
@show x,y,z = (0,2,1)
@show x,y,z

(x, y, z) = [0, 2, 1] = [0, 2, 1]
(x, y, z) = (0, 2, 1) = (0, 2, 1)
(x, y, z) = (0, 2, 1)


(0, 2, 1)

Another simple example of pattern matching are the anonymous functions. 

In [136]:
(x,y) -> x^2 + y^2;

Consider now that we have a dictionary, and that we want to put the respective values in the
varibles. Let's try a similar thing to the previous examples.

In [50]:
d  = Dict(:x=>3, :y=> 2, :z=>1)
x,y,z = d
x,y,z

(:y => 2, :z => 1, :x => 3)

In [53]:
x,typeof(x)

(:y => 2, Pair{Symbol, Int64})

This did not work. For two reasons. First, the `Dict` in Julia does not preserve the order we used in the construction.
Secondly, the varibales actually got Pairs with both key and value.
How do we do this on Base Julia?

In [61]:
for (k,v) in d
    if k == :x
        x = v
    elseif k == :y
        y = v
    elseif k == :z
        z = v
    end
end

x,y,z

(3, 2, 1)

This works. But it's not very elegant, and it's not using pattern matching.
To use pattern matching, we'll use the macro `@match` from `MLStyle`.
Here is how we do it.

In [62]:
x,y,z = @match d begin
    Dict(:x=>x, :y=> y, :z=>z) => [x,y,z]
end

3-element Vector{Int64}:
 3
 2
 1

So what is going on? We just wrote how our dictionary was written, and we matched each variable,
hence the name "pattern matching".
Very neat.

This is also good to keep track of nested structure. Consdider the following tuple. 

In [79]:
t = ("a", "B", ("cc", "d'", (5, )))

("a", "B", ("cc", "d'", (5,)))

Now, suppose you wish to get the values corresponding to positions
1,2,3 and 4.

Here is how to do it without pattern matching.

In [86]:
a = t[1]
b = t[2]
c = t[3][1]
d = t[3][2]

"d'"

This works, but you can easily miss something.
Here is how to do with pattern matching.

In [87]:
@match t begin
           (a, b, (c, d, (5, ))) => (a, b, c, d)
end

("a", "B", "cc", "d'")

Very elegant. Another example. Pattern matching behaves very similar to switch statements. 

In [106]:
check(x)  = @match x begin
    1  => "a"
    10 => "c"
    _  => nothing
end

check (generic function with 2 methods)

In [107]:
check(10),check("b")

("c", nothing)

In [141]:
check(x,y)  = @match (x,y) begin
    (1,1)  => "a"
    (2,1)  => "b"
    if x+y>10 end => "G"
    _ => "No pattern found"
end

check(10,10), check(2,1), check(2,2)

("G", "b", "No pattern found")

### Using Pattern Functions to Manipulate Expressions

We actually already created pattern functions. Our `check()` function is a pattern
function, since it's just a function with a pattern matching inside.

The example below shows how to create an anonymous pattern function.

In [214]:
f = @λ begin
           # patterns here
           ((x, (1, 2)) &&
               if x > 3 end)  => 5
           (x, y)             => 2
           ::String           => "is string"
           _                  => "is any"
end;

This example is similar to: 

In [216]:
f = x -> x^2;

Note that when we do this kind of assignment, the `f` variable is poiting to an annonymous function, which is different
than defining `f` as a function, e.g.

In [219]:
g(x) = x^2

g = 2

LoadError: invalid redefinition of constant g

We can, in a similar fashion, define our patter function:

In [220]:
F(x) = @match x begin
           # patterns here
           ((x, (1, 2)) &&
               if x > 3 end)  => 5
           (x, y)             => 2
           ::String           => "is string"
           _                  => "is any"
end;

Let's show how we can use MLStyle to manipulate more complex patterns
such as functions and structs.

The first issue with expressions (Expr) in Julia is that
we have expressions that represent which line we are. This is useful for
debugging errors, but it won't be of much use to us. Hence, we want to get rid of
it.

Here is an example.

In [221]:
expr = quote
    struct S{T}
        a :: Int
        b :: T
    end
end

quote
    #= In[221]:2 =#
    struct S{T}
        #= In[221]:3 =#
        a::Int
        #= In[221]:4 =#
        b::T
    end
end

Note the `#= In[170]:2 =#`. We want to remove these.

In [290]:
rmlines = @λ begin
    e :: Expr           => Expr(e.head, filter(!isnothing, map(rmlines, e.args))...)
      :: LineNumberNode => nothing
    a                   => a
end;

What is going on here? We created a function called `rmlines`.
This function is recursive. Whenever the input is an `Expr`, we pass it on to `rmlines`
with some tweaking. If it receives a `::LineNumberNode`, it returns `nothing`, which
is then filtered. Otherwise, it spits the value without doing nothing.

In [291]:
expr = quote
    struct S{T}
        a :: Int
        b :: T
    end
end |> rmlines

quote
    struct S{T}
        a::Int
        b::T
    end
end

It works! But how can we use this with pattern matching? Just check the code below:

In [292]:
@match expr begin
    quote
        struct $name{$tvar}
            $f1 :: $t1
            $f2 :: $t2
        end
    end =>
    (name, tvar, f1,t1, f2, t2)
end # true

(:S, :T, :a, :Int, :b, :T)

Let's do another example with a function. 

In [293]:
expr = quote
    function myfunction(x::X,y::Y,z)::T
        dostuff
    end
end |> rmlines

quote
    function myfunction(x::X, y::Y, z)::T
        dostuff
    end
end

In [294]:
@match expr begin
    quote
        function $f($x :: $X,$y :: $Y,$z) :: $T
            $body
        end
    end =>
    (f,body)
end # true

(:myfunction, :dostuff)